In [ ]:
from rapidfuzz import process, fuzz
import unicode

# 한영 자판 변환 딕셔너리
eng_to_kor = {
    'r': 'ㄱ', 'R': 'ㄲ', 's': 'ㄴ', 'e': 'ㄷ', 'E': 'ㄸ', 'f': 'ㄹ', 'a': 'ㅁ', 'q': 'ㅂ', 'Q': 'ㅃ', 't': 'ㅅ',
    'T': 'ㅆ', 'd': 'ㅇ', 'w': 'ㅈ', 'W': 'ㅉ', 'c': 'ㅊ', 'z': 'ㅋ', 'x': 'ㅌ', 'v': 'ㅍ', 'g': 'ㅎ',
    'k': 'ㅏ', 'o': 'ㅐ', 'i': 'ㅑ', 'O': 'ㅒ', 'j': 'ㅓ', 'p': 'ㅔ', 'u': 'ㅕ', 'P': 'ㅖ', 'h': 'ㅗ', 
    'y': 'ㅛ', 'n': 'ㅜ', 'b': 'ㅠ', 'm': 'ㅡ', 'l': 'ㅣ'
}

# 초성 리스트
CHOSUNG_LIST = [
    'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ'
]

HANGUL_BEGIN_UNICODE = 44032
HANGUL_END_UNICODE = 55203
CHOSUNG_BASE = 588

# 초성 추출 함수
def get_chosung(text):
    result = []
    for char in text:
        if HANGUL_BEGIN_UNICODE <= ord(char) <= HANGUL_END_UNICODE:
            chosung_index = (ord(char) - HANGUL_BEGIN_UNICODE) // CHOSUNG_BASE
            result.append(CHOSUNG_LIST[chosung_index])
        else:
            result.append(char)
    return ''.join(result)

# 영어 입력을 한글로 변환하는 함수
def convert_eng_to_kor(eng_text):
    result = []
    for char in eng_text:
        if char in eng_to_kor:
            result.append(eng_to_kor[char])
        else:
            result.append(char)
    return ''.join(result)

# 입력된 텍스트가 초성인지 확인하는 함수
def is_chosung(text):
    return all(char in CHOSUNG_LIST for char in text)


# 상병명 데이터 리스트 (생략된 데이터 포함)
disease_data = []

 # 데이터프레임

# 초성 검색 시 초성이 상병명 초성에 포함되는지 확인하여 검색
def search_with_chosung_inclusion(data, query):
    chosung_query = get_chosung(query)  # 입력된 검색어의 초성 추출
    # 데이터의 초성 추출 후, 초성 문자열에 포함되는지 확인
    matches = [item for item in data if chosung_query in get_chosung(item)]
    return matches

def search_with_partial_and_correction(data, query, threshold=50):
    # 1. 부분 일치를 먼저 확인하고, 유사도가 높은 상병명 선택
    matches = process.extract(query, data, scorer=fuzz.partial_ratio, limit=15)
    return [match[0] for match in matches if match[1] >= threshold]

# 검색 실행 함수
def search_with_options(data, query):
    # 초성만 입력된 경우
    if is_chosung(query):
        print("초성 검색을 실행합니다.")
        return search_with_chosung_inclusion(data, query)
    
    # 영어로 입력된 경우 한글로 변환
    if all(char.isalpha() and char.islower() for char in query):
        print("영어 입력을 한글로 변환합니다.")
        query = unicode.join_jamos(convert_eng_to_kor(query))
        print(query)
    
    # 부분 일치 및 오타 보정
    return search_with_partial_and_correction(data, query)

# 검색어
search_query = "급성 염"

# 검색 실행
result = search_with_options(disease_data, search_query)

# 결과 출력
print("검색어:", search_query)
print("검색 결과:", result)